In [1]:
import os
import io
#Cred key
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../1_Vision/Keys/glossy-radio-377415-d797ca03c624.json'

In [5]:
image_path = os.path.abspath('./image02.jpg')

In [6]:
def localize_objects(path):
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()
    
    #open local file for detection
    with open(path, 'rb') as image_file:
        content = image_file.read()
    
    image = vision.Image(content=content)
    objects = client.object_localization(image=image).localized_object_annotations
    print('Number of objects found: {}'.format(len(objects)))
    
    #PIL is used to crop-out new files from original image
    from PIL import Image
    for object in objects:
        detected = []
        limits = []
        print('\n{} (Confidence: {})'.format(object.name, object.score))
        for vertex in object.bounding_poly.normalized_vertices:
            print(' - ({}, {})'.format(vertex.x, vertex.y))
            limits.append((vertex.x, vertex.y))
        detected.append(limits)
    return detected

In [7]:
boxes = localize_objects(image_path)

Number of objects found: 3

Person (Confidence: 0.9051889181137085)
 - (0.033025432378053665, 0.195871964097023)
 - (0.9207284450531006, 0.195871964097023)
 - (0.9207284450531006, 0.9972599148750305)
 - (0.033025432378053665, 0.9972599148750305)

Flower (Confidence: 0.7622988224029541)
 - (0.5749930739402771, 0.8048368096351624)
 - (0.8495102524757385, 0.8048368096351624)
 - (0.8495102524757385, 0.9949901103973389)
 - (0.5749930739402771, 0.9949901103973389)

Clothing (Confidence: 0.688556969165802)
 - (0.008878236636519432, 0.7053647041320801)
 - (0.8963396549224854, 0.7053647041320801)
 - (0.8963396549224854, 0.9945178031921387)
 - (0.008878236636519432, 0.9945178031921387)


In [8]:
print(boxes)

[[(0.008878236636519432, 0.7053647041320801), (0.8963396549224854, 0.7053647041320801), (0.8963396549224854, 0.9945178031921387), (0.008878236636519432, 0.9945178031921387)]]


In [9]:
from PIL import Image
orig_image = Image.open(image_path)
x = boxes[0][0][0] * orig_image.width
y = boxes[0][0][1] * orig_image.height
x2 = boxes[0][2][0] * orig_image.width
y2 = boxes[0][2][1] * orig_image.height
obj_image = orig_image.crop((x, y, x2, y2))
obj_image.save("object-3.png")